In [2]:
from pydantic import BaseModel, Field
from typing import Optional
from langchain.chat_models import ChatOpenAI
from langchain.chains.structured_output import create_openai_fn_runnable, create_structured_output_runnable

### 2 a. `create_openai_fn_runnable`

In [5]:
class ExtractCommentaryDetails(BaseModel):
    '''Extract details from cricket commentary.'''
    player_name: str = Field(..., description="The name of the players")
    action: str = Field(..., description="The name of the shot played")
    score: Optional[int] = Field(None, description="Score achieved by the player")
    critique: Optional[str] = Field(None, description="The commentator's adjectives for the player")

llm = ChatOpenAI(api_key=API_KEY, model="gpt-4", temperature=0)
structured_llm = create_openai_fn_runnable([ExtractCommentaryDetails], llm)

commentary = "Ponting faces the ball, and hits a cover drive for a four, right above Rhodes' head! That went like a bullet!"

result = structured_llm.invoke(commentary)
print(result)

{'player_name': 'Ponting', 'action': 'hits a cover drive for a four', 'score': 4, 'critique': 'That went like a bullet!'}


### 2 b. `create_structured_output_runnable`

In [7]:
class BookReader(BaseModel):
    '''Identifying information about a research paper.'''
    title: str = Field(..., description="The title of the book")
    authors: str = Field(..., description="The authors of the book")
    publication_year: int = Field(..., description="The year the book was published")
    publisher: str = Field(..., description="The publisher of the book")
    other_details: Optional[str] = Field(None, description="Other details about the book")

llm = ChatOpenAI(api_key=API_KEY, model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = create_structured_output_runnable(BookReader, llm, mode="openai-tools", enforce_function_usage=True, return_single=True)

input_text = """
"Statistical Inference" was written by George Casella and Roger Berger in 2001 by Cengage. The book starts with a quote from Sherlock Holmes."""

result = structured_llm.invoke(input_text)
print(result)

{'title': 'Statistical Inference', 'authors': 'George Casella and Roger Berger', 'publication_year': 2001, 'publisher': 'Cengage', 'other_details': 'The book starts with a quote from Sherlock Holmes.'}


### 2 c. `create_stuff_documents_chain`

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders import PyMuPDFLoader

# Load PDF document
loader = PyMuPDFLoader("alphago_paper.pdf")
docs = loader.load()

prompt = ChatPromptTemplate.from_messages(
    [("system", "Who wrote the paper:\n\n{context}")]
)
llm = ChatOpenAI(api_key=API_KEY, model="gpt-3.5-turbo")
chain = create_stuff_documents_chain(llm, prompt)

result = chain.invoke({"context": docs})
print(result)

The paper "Mastering the Game of Go without Human Knowledge" was written by David Silver, Julian Schrittwieser, Karen Simonyan, Ioannis Antonoglou, Aja Huang, Arthur Guez, Thomas Hubert, Lucas Baker, Matthew Lai, Adrian Bolton, Yutian Chen, Timothy Lillicrap, Fan Hui, Laurent Sifre, George van den Driessche, Thore Graepel, and Demis Hassabis from DeepMind.


### 3. Custom chain using components

In [10]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.output_parsers.structured import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Memory store for creating documents on the fly
vectorstore = DocArrayInMemorySearch.from_texts(
    ["Jett is a duelist who can updraft and throw knives", "Omen is a controller who can create smokes and teleport"],
    embedding=OpenAIEmbeddings(openai_api_key=API_KEY),
)
retriever = vectorstore.as_retriever()

# Template for creating structured output based on "context" and "question" variables
template = """Answer the question based only on the following context:
{context}

Question: {question}

Please format the answer as a JSON object with a single key "answer"."""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=API_KEY)

# Define the response schema
response_schemas = [
    ResponseSchema(name="answer", description="The answer to the question", type="string")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Define the setup and retrieval function using RunnableParallel
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

# Create the full chain
chain = setup_and_retrieval | prompt | model | output_parser

# Invoking the chain with a Valorant-related question
result = chain.invoke("What type of agent is Jett?")

print(result['answer'])


duelist
